In [41]:
import sys
sys.path.append("/Users/kim/Desktop/repos/Philippines_Visitor")

import pandas as pd
from db_conn import get_engine

query = """
SELECT
    year, 
    sum(total_visitors) as Total_Visitors
FROM 
    yearly_visitors yv 
GROUP BY year
ORDER BY year;
"""

def get_visitors_per_year():
    engine = get_engine()
    df = pd.read_sql(query, engine)
    return df


In [46]:
visitors_per_year_df = get_visitors_per_year()
visitors_per_year_df.to_csv("updated_visitorperyear.csv")

In [43]:
visitors_per_year_df['Growth Rate (%)'] = visitors_per_year_df['total_visitors'].pct_change().mul(100).round(2)
visitors_per_year_df

,year,total_visitors,Growth Rate (%)
0,2015,5148819.0,NaN
1,2016,5776582.0,12.19
2,2017,6480297.0,12.18
3,2018,7096594.0,9.51
4,2019,8188477.0,15.39
5,2020,1398455.0,-82.92
6,2021,146098.0,-89.55
7,2022,2025413.0,1286.34
8,2023,4991947.0,146.47
9,2024,5438967.0,8.95


In [44]:
baseline_2019 = visitors_per_year_df.loc[visitors_per_year_df['year'] == 2019, 'total_visitors'].values[0]
baseline_2019

visitors_per_year_df['recovery_index'] = (visitors_per_year_df['total_visitors'] / baseline_2019) * 100

visitors_per_year_df.to_csv("Visitors per Year_Recovery.csv")

In [45]:
visitors_per_year_df

,year,total_visitors,Growth Rate (%),recovery_index
0,2015,5148819.0,NaN,62.878836
1,2016,5776582.0,12.19,70.545255
2,2017,6480297.0,12.18,79.139222
3,2018,7096594.0,9.51,86.665616
4,2019,8188477.0,15.39,100.000000
5,2020,1398455.0,-82.92,17.078328
6,2021,146098.0,-89.55,1.784190
7,2022,2025413.0,1286.34,24.734917
8,2023,4991947.0,146.47,60.963070
9,2024,5438967.0,8.95,66.422205


In [37]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


fig = go.Figure()

# Create subplot: 2 rows, shared x-axis
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.15,
    subplot_titles=("Total Visitors per Year", "Recovery Index (2019 = 100)")
)

# Top chart: Total Visitors
fig.add_trace(
    go.Bar(
        x=visitors_per_year_df['year'],
        y=visitors_per_year_df['total_visitors'],
        name='Total Visitors',
        customdata=visitors_per_year_df[['Growth Rate (%)']],
        hovertemplate=
            'Year: %{x}<br>Total Visitors: %{y:,.0f}<br>YoY Growth: %{customdata[0]:.2f}%<extra></extra>'
    ),
    row=1, col=1
)

# Bottom chart: Recovery Index
fig.add_trace(
    go.Scatter(
        x=visitors_per_year_df['year'],
        y=visitors_per_year_df['recovery_index'],
        name='Recovery Index',
        mode='lines+markers',
        hovertemplate='Year: %{x}<br>Recovery: %{y:.1f}% of 2019<extra></extra>',
        fill='tozeroy'
    ),
    row=2, col=1
)

# Shade COVID impact
fig.add_shape(
    type='rect',
    x0=2020, x1=2021,
    y0=0, y1=visitors_per_year_df['recovery_index'].max()*1.1,
    fillcolor='yellow',
    opacity=0.2,
    layer='below',
    line_width=0
)


# Layout
fig.update_layout(
    height=700,
    title_text="Philippines Tourism (2015–2025): Volume and Recovery",
    template='plotly_white'
)

# Axis labels
fig.update_yaxes(title_text="Total Visitors", row=1, col=1)
fig.update_yaxes(title_text="Recovery Index", row=2, col=1)
fig.update_xaxes(title_text="Year", row=2, col=1)

fig.show()